In [1]:
# Install the very latest Segment Anything (v2) and its deps
!pip install --upgrade git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python-headless numpy


  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-_cpdu714
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-_cpdu714
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36592 sha256=b82a8052c7c0fd29bed7baea7b43638ec5298b02b18beb312112cba13ec9c2fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-57odlubi/wheels/15/d7/bd/05f5f23b7dcbe70cbc6783b06f12143b0cf1a5da5c7b52dcc5
Successfully built segment_anything


In [2]:
!pip install huggingface_hub


In [3]:
!wget -O sam_vit_b_01ec64.pth \
    https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


--2025-05-24 06:34:32--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.15, 13.226.210.78, 13.226.210.25, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.15|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 375042383 (358M) [binary/octet-stream]
Saving to: ‘sam_vit_b_01ec64.pth’

sam_vit_b_01ec64.pt 100%[===================>] 357.67M  8.17MB/s    in 5.5s    

2025-05-24 06:34:37 (65.6 MB/s) - ‘sam_vit_b_01ec64.pth’ saved [375042383/375042383]



In [5]:
from google.colab import files
uploaded = files.upload()


Saving image_1.jpg to image_1.jpg
Saving image_2.jpg to image_2.jpg
Saving image_3.jpg to image_3.jpg
Saving image_4.jpg to image_4.jpg
Saving image_5.jpg to image_5.jpg
Saving image_6.jpg to image_6.jpg
Saving image_7.jpg to image_7.jpg
Saving image_8.jpg to image_8.jpg
Saving image_9.jpg to image_9.jpg
Saving image_10.jpg to image_10.jpg
Saving image_11.jpg to image_11.jpg
Saving image_12.jpg to image_12.jpg
Saving image_13.jpg to image_13.jpg
Saving image_14.jpg to image_14.jpg
Saving image_15.jpg to image_15.jpg
Saving image_16.jpg to image_16.jpg
Saving image_17.jpg to image_17.jpg
Saving image_18.jpg to image_18.jpg
Saving image_19.jpg to image_19.jpg
Saving image_20.jpg to image_20.jpg
Saving image_21.jpg to image_21.jpg
Saving image_22.jpg to image_22.jpg
Saving image_23.jpg to image_23.jpg
Saving image_24.jpg to image_24.jpg
Saving image_25.jpg to image_25.jpg
Saving image_26.jpg to image_26.jpg
Saving image_27.jpg to image_27.jpg
Saving image_28.jpg to image_28.jpg
Saving ima

In [6]:
import os, cv2, csv, numpy as np, torch
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator

# 1. Use current directory for uploaded images
IMAGE_DIR = "/content"
OUTPUT_CSV = "/content/sam_output_boxes.csv"

# 2. Choose device
device = "cuda" if torch.cuda.is_available() else "cpu"

# 3. Load SAM model (update path to checkpoint if needed)
sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth").to(device)

# 4. Initialize mask generator
mask_generator = SamAutomaticMaskGenerator(
    model=sam,
    points_per_side=40,
    points_per_batch=64,
    pred_iou_thresh=0.90,
    stability_score_thresh=0.92,
    box_nms_thresh=0.6
)

# 5. Prepare CSV
with open(OUTPUT_CSV, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["image_name", "box_id", "x_min", "y_min", "x_max", "y_max"])

    # 6. Process images in folder
    for fname in sorted(os.listdir(IMAGE_DIR)):
        if not fname.lower().endswith((".png", ".jpg", ".jpeg", ".tif")):
            continue

        path = os.path.join(IMAGE_DIR, fname)
        img = cv2.imread(path)
        if img is None:
            print(f"⚠️ Cannot read image: {fname}")
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        masks = mask_generator.generate(img_rgb)
        print(f"{fname}: {len(masks)} masks generated")

        # Filter masks by area
        filtered = [m for m in masks if 3500 < m["segmentation"].sum() < 9000]
        print(f"   → {len(filtered)} masks kept after filtering")

        # Save bounding boxes to CSV
        for i, m in enumerate(filtered):
            x, y, w, h = m["bbox"]
            x_max = x + w
            y_max = y + h
            writer.writerow([fname, i, int(x), int(y), int(x_max), int(y_max)])

print("✅ Done! CSV saved at:", OUTPUT_CSV)


image_1.jpg: 28 masks generated
   → 3 masks kept after filtering
image_10.jpg: 33 masks generated
   → 17 masks kept after filtering
image_100.jpg: 15 masks generated
   → 6 masks kept after filtering
image_101.jpg: 15 masks generated
   → 5 masks kept after filtering
image_102.jpg: 20 masks generated
   → 3 masks kept after filtering
image_103.jpg: 14 masks generated
   → 3 masks kept after filtering
image_104.jpg: 16 masks generated
   → 4 masks kept after filtering
image_105.jpg: 10 masks generated
   → 2 masks kept after filtering
image_106.jpg: 19 masks generated
   → 8 masks kept after filtering
image_107.jpg: 26 masks generated
   → 9 masks kept after filtering
image_108.jpg: 12 masks generated
   → 1 masks kept after filtering
image_109.jpg: 16 masks generated
   → 2 masks kept after filtering
image_11.jpg: 34 masks generated
   → 21 masks kept after filtering
image_110.jpg: 12 masks generated
   → 2 masks kept after filtering
image_111.jpg: 14 masks generated
   → 6 masks kep

In [7]:
from google.colab import files
files.download("/content/sam_output_boxes.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>